In [1]:
from google.cloud import bigquery
import pandas as pd
import os
import time

os.environ['GOOGLE_APPLICATION+CREDENTIALS'] = '/Users/nrabbani/.config/gcloud/application_default_credentials.json'
os.environ['GCLOUD_PROJECT'] = 'mining-clinical-decisions'
%load_ext google.cloud.bigquery

client=bigquery.Client()

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For mor

In [3]:
query = '''
WITH labs AS (
    SELECT * FROM njr_labutil.lradt
    WHERE
        (pat_class = 'Inpatient' OR pat_class = 'Surgery Admit')
        AND pat_service NOT IN (
            'Obstetrics', 'Maternal-Fetal Medicine',
            'Reproductive Endocrinology and Infertility',
            'Gynecology & Obstetrics', 'Gynecology', 'OB Anesthesia',
            'Intensive Care', 'Neonatology', 'Cardiovascular Intensive Care')
        AND (pat_lv_of_care = 'Acute Care'
            OR (pat_lv_of_care='' AND accomodation IN (
                'PEDS MED/SURG Level 1','PEDS MED/SURG Level 2', 'PEDS MED/SURG Level 3',
                'PEDS Med/Surg',
                'CCP Med/Surg Level 1','CCP Med/Surg Level 2','CCP Med/Surg Level 3',
                'PEDS TXP Med/Surg Level 1', 'PEDS TXP Med/Surg Level 2','PEDS TXP Med/Surg Level 3',
                'Oncology Level 1', 'Oncology Level 2', 'Oncology Level 3', 'Oncology Level 4'
                )
            )
        )
)
SELECT group_lab_name, COUNT(group_lab_name) cnt, COUNT(DISTINCT anon_id) pts FROM (
    SELECT DISTINCT(order_id_coded), anon_id, group_lab_name
    FROM labs
)
GROUP BY(group_lab_name)
ORDER by cnt desc
'''

In [4]:
query_job = client.query(query)
df = query_job.to_dataframe()

In [30]:
df.head(30)

,group_lab_name,cnt,pts
0,CBC with Differential,57451,6319
1,Chem 10,47573,4601
2,Glucose PCX Meter,43000,2094
3,"Magnesium, Ser/Plas",42457,4643
4,Renal Function Panel,25686,3073
5,"Metabolic Panel, Comprehensive",23308,4825
6,"Phosphorus, Ser/Plas",21245,3333
7,Urinalysis Macroscopic Only,19734,1004
8,Manual Differential/Slide Review,18973,2189
9,ABO Rh Typing,16377,2544


In [27]:
query_all = '''
    SELECT * FROM njr_labutil.lradt
    WHERE
        (pat_class = 'Inpatient' OR pat_class = 'Surgery Admit')
        AND pat_service NOT IN (
            'Obstetrics', 'Maternal-Fetal Medicine',
            'Reproductive Endocrinology and Infertility',
            'Gynecology & Obstetrics', 'Gynecology',
            'Intensive Care', 'Neonatology', 'Cardiovascular Intensive Care')
        AND (pat_lv_of_care = 'Acute Care'
            OR (pat_lv_of_care='' AND accomodation IN (
                'PEDS MED/SURG Level 1','PEDS MED/SURG Level 2', 'PEDS MED/SURG Level 3',
                'PEDS Med/Surg',
                'CCP Med/Surg Level 1','CCP Med/Surg Level 2','CCP Med/Surg Level 3',
                'PEDS TXP Med/Surg Level 1', 'PEDS TXP Med/Surg Level 2','PEDS TXP Med/Surg Level 3',
                'Oncology Level 1', 'Oncology Level 2', 'Oncology Level 3', 'Oncology Level 4'
                )
            )
        )
'''
query_job = client.query(query_all)

In [28]:
tic = time.time()
lradt = query_job.to_dataframe()
print(time.time() - tic)

1353.993928194046


In [29]:
lradt.loc[~lradt.ord_num_value.isnull()].to_pickle('data/lradt_20210927.pkl')

In [ ]:
query_accom = '''
SELECT * FROM njr_labutil.lradt
WHERE
    (pat_class = 'Inpatient' OR pat_class = 'Surgery Admit')
    AND pat_service NOT IN (
        'Obstetrics', 'Maternal-Fetal Medicine',
        'Reproductive Endocrinology and Infertility',
        'Gynecology & Obstetrics', 'Gynecology',
        'Intensive Care', 'Neonatology', 'Cardiovascular Intensive Care'
    )
    AND (pat_lv_of_care = 'Acute Care' OR pat_lv_of_care = '')
    AND accomodation IN (
            'PEDS MED/SURG Level 1','PEDS MED/SURG Level 2', 'PEDS MED/SURG Level 3',
            'PEDS Med/Surg',
            'CCP Med/Surg Level 1','CCP Med/Surg Level 2','CCP Med/Surg Level 3'
    )

'''
query_job = client.query(query_accom)
lradt_acc = query_job.to_dataframe()
lradt_acc.loc[~lradt.ord_num_value.isnull()].to_pickle('data/lradt_acc_20210927.pkl')

In [3]:
lradt_acc.loc[~lradt_acc.ord_num_value.isnull()].to_pickle('data/lradt_acc_20210927.pkl')